Proyecto (primera parte) 

En esta primera parte del proyecto se estudiará e implementará el algoritmo de Búsqueda Lineal de Newton con modificación a la Hessiana. 


In [1]:
#Importar librerías 
import numpy as np
import math as ma
import random as rd
from numpy import linalg as la

In [2]:
#Código para caluclar el gradiente
#elegimos un error apropiado
def grad(f,xo):
    n = len(xo)
    eps=0.000001
    res=np.zeros(n)
    for i in range(n):
        zer = np.zeros(n)
        zer[i] += eps
        x1 = xo + zer
        res[i] = (f(x1)-f(xo))/eps
    return res

In [3]:
#Aquí definimos nuestras funciones para probar nuestro código
#función 1
def i(xo):
    suma=0
    for i in range(len(xo)):
        suma=suma+ma.sqrt(xo[i])
    return suma

xo = np.array([4,2,5,2])
#imprimimos el valor de la función en xo 
i(xo)

7.06449510224598

In [34]:
#Definimos la función Rosenbrock
xo=[1,1]
def rosen(xo):
    return 100*(xo[1]-(xo[0])**2)**2+(1-xo[0])**2

def rosen_1(xo):
    return 20*(xo[1]-(xo[0])**2)**2+(2-xo[0])**2

def rosen_2(xo):
    return 50*(xo[1]-(xo[0])**2)**2+(0-xo[0])**2
#imprimimos su valor en ese punto 



In [35]:
#Calculamos el gradiente de forma anlítica y lo calcula bien 
grad(rosen,xo)

array([4.010004e-04, 1.000000e-04])

In [36]:
#Código que calcula la hessiana
#el desarrollo de este método se hará con diferencias finitas 
def hess(f,xo):
    n = len(xo)
    eps=0.000001
    res=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            zer = np.zeros(n)
            zer2 = np.zeros(n)
            zer[i] += eps
            zer2[j] += eps
            x_e = xo + zer + zer2
            x_ei = xo + zer
            x_ej = xo + zer2
            res[i][j] = (f(x_e)-f(x_ei)-f(x_ej)+f(xo))/(eps**2)
    return res

In [37]:
#imprimimos la Hessiana 
hess(rosen,xo)

array([[ 802.00239973, -400.00019995],
       [-400.00019995,  199.99999997]])

In [38]:
#Código que regresa si el punto xk cumple con las condiciones de optimalidad
def condiciones_optimalidad(f,xo,tol):
    n = len(grad(f,xo))
    v,w = la.eig(hess(f,xo))
    n1 = len(v)
    bandera = True
    bandera1 = True
    eps=0.0001
    
    for i in range(n):
        if abs(grad(f,xo)[i])>tol:
            bandera= False
        if(v[i]<=0):
            bandera1 = False
            
    if (bandera == True and bandera1 == True):
        return True
    else:
        return False
    

In [39]:
#permitimos que el usuario ingrese un nivel de tolerancia 
condiciones_optimalidad(rosen,xo,.001)

True

In [40]:
# definir función mk 
def mk(f,xo,p):
    H = hess(f,xo)
    G = grad(f,xo)
    pt= np.transpose(p)
    return f(xo) + np.dot(pt,G) + 0.5*np.dot(np.dot(pt,H),pt)

In [41]:
#probamos con un punto la función
p=[.01,.02]
mk(rosen,xo,p)

0.00010608999340135658

In [42]:
#Algoritmo de búsqueda de paso (BacksS)
def BackS(a,f,xk,pk):
    rd.seed(123)
    c = rd.uniform(0,1)
    while f(xk+a*pk)>f(xk)+c*a*np.dot(grad(f,xk),pk):
        rho = rd.uniform(0,1)
        a = rho*a
    return a 


In [43]:
BackS(1,rosen,np.array([1.2,1.2]),np.array([-1,-2]))

0.08718667752263232

In [44]:
#Algoritmo Cholesky con mútilplo de la identidad
#np.transpose
def Cholesky(A,b,k):
    t=0
    if min(np.diag(A)) > 0:
         t = 0
    else: 
        t = -min(np.diag(A)) + b
        
    for j in range(k):
        try: 
            L=la.cholesky(A + t*np.identity(len(A)))
        except:
            t = max(2*t,b)
        else:
            break
    return np.dot(L,L)
    

In [45]:
#Algoritmo de búsqueda lineal de Newton Modificado
def Newton_Mod(xk,f,n,k,b):
    for i in range(n):
        Bk = hess(f,xk)
        try:
            L=la.cholesky(Bk)
        except:
            Bk=Cholesky(Bk,b,k)
        pk = np.dot(la.inv(Bk),-1*grad(f,xk))
        xk=xk+BackS(1,f,xk,pk)*pk
    return xk
        

In [46]:
Newton_Mod([-1.2,1],rosen,1000,1000,1)

array([0.99969965, 0.99939889])

In [47]:
Newton_Mod([1.2,1.2],rosen_1,1000,1000,1)

array([1.99979954, 3.99919768])

In [33]:
Newton_Mod([1.2,1.2],rosen_2,1000,1000,1)

array([-4.54910459e-07, -5.00131355e-07])